In [1]:
import requests
import bs4

In [2]:
response = requests.get('http://pyvideo.org/category/50/pycon-us-2014')

In [3]:
type(response)

requests.models.Response

In [4]:
response.headers

{'content-encoding': 'gzip', 'transfer-encoding': 'chunked', 'set-cookie': 'csrftoken=iuZN5fqgZmJkdo4Lg0A4FGfLcTaGlMZy; expires=Mon, 23-May-2016 18:35:33 GMT; Max-Age=31449600; Path=/', 'vary': 'Cookie', 'server': 'nginx/1.4.6 (Ubuntu)', 'connection': 'keep-alive', 'date': 'Mon, 25 May 2015 18:35:33 GMT', 'content-type': 'text/html; charset=utf-8'}

In [5]:
type(response.text)

unicode

In [6]:
response.text[0:200]

u'\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n<head>\n  <link rel="stylesheet" href="/static/bootstrap-3.1.1/css/bootstrap.min.css">\n  <link rel="stylesheet" href="/static/css/richard.css">\n  <link rel="style'

In [7]:
soup = bs4.BeautifulSoup(response.text)



/home/cma/anaconda/lib/python2.7/site-packages/bs4/dammit.py:269: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if (len(data) >= 4) and (data[:2] == b'\xfe\xff') \
/home/cma/anaconda/lib/python2.7/site-packages/bs4/dammit.py:273: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  elif (len(data) >= 4) and (data[:2] == b'\xff\xfe') \
/home/cma/anaconda/lib/python2.7/site-packages/bs4/dammit.py:277: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  elif data[:3] == b'\xef\xbb\xbf':
/home/cma/anaconda/lib/python2.7/site-packages/bs4/dammit.py:280: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  elif data[:4] == b'\x00\x00\xfe\xff':
/home/cma/anaconda/lib/python2.7/site-packages/bs4/dammit.py:2

In [8]:
links0  = soup.find_all('a',href=True)

In [9]:
#for a in links0:
#   if 'video' in a['href']:
#      print a

In [10]:
links = soup.select('a[href^=/video]')

In [11]:
links2 = soup.find('div',{'id':'video-summary-content'}).findAll('a')

In [12]:
type(links2)

bs4.element.ResultSet

In [13]:
len(links2)

44

In [14]:
links3=[a.attrs.get('href') for a in soup.find('div',{'id':'video-summary-content'}).findAll('a') ]

In [15]:
links4=[a.attrs.get('href') for a in soup.select('div.col-md-3 a[href^=/video]') ]

In [16]:
len(links4)

11

In [17]:
len(links3)

44

In [18]:
root_url = 'http://pyvideo.org'
index_url = root_url + '/category/50/pycon-us-2014'
def get_video_page_urls():
    response = requests.get(index_url)
    soup = bs4.BeautifulSoup(response.text)
 #   return [a.attrs.get('href') for a in soup.find('div',{'id':'video-summary-content'}).findAll('a') ]
    return [a.attrs.get('href') for a in soup.select('div.col-md-3 a[href^=/video]') ]

In [19]:
response = requests.get('http://pyvideo.org//video/2668/writing-restful-web-services-with-flask')

In [20]:
soup = bs4.BeautifulSoup(response.text)

In [21]:
soup.select('div#videobox h3')[0]

<h3 class="centered" property="name">Writing RESTful web services with Flask</h3>

In [22]:
response = requests.get('https://www.youtube.com/watch?v=px_vg9Far1Y')

In [23]:
soup = bs4.BeautifulSoup(response.text)

In [24]:
soup.select('.watch-view-count')[0].get_text().split()

IndexError: list index out of range

In [ ]:
import re


In [ ]:
re.sub('[^0-9]', '',soup.select('.watch-view-count')[0].get_text().split()[0])

In [ ]:
def get_video_data(video_page_url):
        video_data = {}
        response = requests.get(root_url + video_page_url)
        soup = bs4.BeautifulSoup(response.text)
        video_data['title'] = soup.select('div#videobox h3')[0].get_text()
        video_data['speakers'] = [a.get_text() for a in soup.select('div#sidebar a[href^=/speaker]')]
 
        # initialize counters
        video_data['views'] = 0
        video_data['likes'] = 0
        video_data['dislikes'] = 0
 
        try:
            video_data['youtube_url'] = soup.select('div#sidebar a[href^=http://www.youtube.com]')[0].get_text()
            response = requests.get(video_data['youtube_url'], headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36'})
            soup = bs4.BeautifulSoup(response.text)
            video_data['views'] = int(re.sub('[^0-9]', '',
                                         soup.select('.watch-view-count')[0].get_text().split()[0]))
            video_data['likes'] = int(re.sub('[^0-9]', '',
                                             soup.select('#watch-like-dislike-buttons span.yt-uix-button-content')[0].get_text().split()[0]))
            video_data['dislikes'] = int(re.sub('[^0-9]', '',
                                            soup.select('#watch-like-dislike-buttons span.yt-uix-button-content')[2].get_text().split()[0]))
        except:
        # some or all of the counters could not be scraped
            pass
        return video_data


In [ ]:
from multiprocessing import Pool

def show_video_stats():
    #pool = Pool(8)
    #video_page_urls = get_video_page_urls()
    #results = pool.map(get_video_data, video_page_urls)
    
    pool = Pool(4)
    video_page_urls = get_video_page_urls()
    results = sorted(pool.map(get_video_data, video_page_urls), key=lambda video: video[views],
                     reverse=True)
    print len(results)
    max = 25
    if max is None or max > len(results):
        max = len(results)
    if options.csv:
        print(u'"title","speakers", "views","likes","dislikes"')
    else:
        print(u'Views  +1  -1 Title (Speakers)')
    for i in range(max):
        if options.csv:
            print(u'"{0}","{1}",{2},{3},{4}'.format(
                results[i]['title'], ', '.join(results[i]['speakers']), results[i]['views'],
                results[i]['likes'], results[i]['dislikes']))
        else:
            print(u'{0:5d} {1:3d} {2:3d} {3} ({4})'.format(
                results[i]['views'], results[i]['likes'], results[i]['dislikes'], results[i]['title'],
                ', '.join(results[i]['speakers'])))


In [ ]:
def parse_args():
    parser = argparse.ArgumentParser(description='Show PyCon 2014 video statistics.')
    parser.add_argument('--sort', metavar='FIELD', choices=['views', 'likes', 'dislikes'],
                        default='views',
                        help='sort by the specified field. Options are views, likes and dislikes.')
    parser.add_argument('--max', metavar='MAX', type=int, default=20, help='show the top MAX entries only.')
    parser.add_argument('--csv', action='store_true', default=False,
                        help='output the data in CSV format.')
    parser.add_argument('--workers', type=int, default=4,
                        help='number of workers to use, 8 by default.')
    return parser.parse_args()


In [ ]:
import argparse

In [ ]:
show_video_stats(parse_args()) --sort views  --max 25 --workers 4

In [25]:
%run -t /var/tmp/git/python.data.analysis.video/gist.course/pycon-views.py --sort views --max 25 --workers 4


11
Views  +1  -1 Title (Speakers)
    0   0   0 2D/3D graphics with Python on mobile platforms (Niko Skrypnik)
    0   0   0 Advanced techniques for Web functional testing (Julien Phalip)
    0   0   0 Analyzing Rap Lyrics with Python (Julie Lavoie)
    0   0   0 A Scenic Drive through the Django Request-Response Cycle (Dan Langer)
    0   0   0 Building the App (Mike Bayer)
    0   0   0 Cheap Helicopters In My Living Room (Ned Jackson Lovely)
    0   0   0 Closing address - PyCon 2014 (2014/04/13) ()
    0   0   0 Data intensive biology in the cloud: instrumenting ALL the things (C. Titus Brown)
    0   0   0 Deliver Your Software In An Envelope (Augie Fackler, Nathaniel Manista)
    0   0   0 Designing Django's Migrations (Andrew Godwin)
    0   0   0 Designing Poetic APIs (Erik Rose)

IPython CPU timings (estimated):
  User   :       1.37 s.
  System :       0.34 s.
Wall time:      69.40 s.
